In [1]:
import flapjack
from flapjack import registry as reg
from flapjack import analysis as analysis
import matplotlib.pyplot as plt
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import scipy
from scipy.optimize import curve_fit
%matplotlib inline

In [2]:
registry = reg.Registry(database_name='data', username='timrudge', password='1234')

We can now get a pandas dataframe with all the data for a particular plasmid:

In [22]:
j = registry.join_all()
plasmid_query = j.filter(flapjack.tables.plasmids.name=='pLux76:RFP/pLacI:YFP/std:CFP') \
        .filter(flapjack.tables.samples.media=='M9-glucosa')
all_data = psql.read_sql_query(j.selectable, registry.engine)
pLuxpLacStd_gluc = psql.read_sql_query(plasmid_query.selectable, registry.engine)
measurement_names = np.unique(pLuxpLacStd_gluc['measurements_name'].values)  #command to get sample id values 
max_values = {}
for m in measurement_names:
    max_values[m] = np.max(all_data['measurements_value'].values)

In [23]:
mx1 = max_values['CFP:420/50,485/20']
mx2 = max_values['RFP-YFP:500/27,540/25']
print analysis.channel_entropy(pLuxpLacStd_gluc, 'CFP:420/50,485/20', 32, [0,mx1])
print analysis.channel_entropy(pLuxpLacStd_gluc, 'RFP-YFP:500/27,540/25', 32, [0,mx1])
print analysis.channel_conditional_entropy(pLuxpLacStd_gluc, \
                                           'CFP:420/50,485/20', \
                                           'RFP-YFP:500/27,540/25', \
                                           [32,32], [[0,mx1],[0,mx2]])

1.2293169669831963
3.0979769957868406
1.980043128318828


Pandas allows us to filter, select, group and otherwise manipulate the data set, and provides some plotting functions:

In [ ]:
yfp_glicerol = df[(df.measurements_name=='RFP-YFP:500/27,540/25')&(df.samples_media=='M9-glicerol')]
cfp_glicerol = df[(df.measurements_name=='CFP:420/50,485/20')&(df.samples_media=='M9-glicerol')]
od_glicerol = df[(df.measurements_name=='OD600:600')&(df.samples_media=='M9-glicerol')]

yfp_glicerol.plot.scatter(x='measurements_time', y='measurements_value')

Selection of a specific experiment (skip if all data required)

In [ ]:
yfp_glic=yfp_glicerol[(yfp_glicerol.samples_experiment_id==4)]
cfp_glic=cfp_glicerol[(cfp_glicerol.samples_experiment_id==4)]
od_glic=od_glicerol[(od_glicerol.samples_experiment_id==4)]

Pandas data series can be directly plotted using Matplotlib/Pyplot:

In [ ]:
plt.figure()
plt.plot(yfp_glic['measurements_value'], cfp_glic['measurements_value'], '.')
plt.xlabel('yfp (AU)')
plt.ylabel('cfp (AU)')

In the end for complete analysis we can get simple numpy arrays by using the `values` attribute, for example:

In [ ]:
y = yfp_glic['measurements_value'].values
c = cfp_glic['measurements_value'].values
t = cfp_glic['measurements_time'].values
od = od_glic['measurements_value'].values
xy = np.log(y)
xc = np.log(c)
plt.plot(c,y,'.')
plt.xlabel('ln(yfp) (AU)')
plt.ylabel('ln(cfp) (AU)')

There are different models that can be used to fit the data. Each one has different inputs and outputs so keep that in mind for the code.

In [ ]:
#Models
t=od_glic['measurements_time'].values  #auxiliary variable, used as a substitute for all time values
print(t.shape,t[700])

def poly(t, mu, sig):                  #polymerase concentration simulation (parameters=mean,std dev)
    P = np.exp(-(t-mu)**2/(2*sig**2))
    return(P)

def I(X, mu, sig, Klcm):           # Model for total fluorescence (input=od,time; 
                                       # parameters=mean,std dev,promoter affinity,translation rate*max transcription rate;
    A,t=X                              # output=Theoretical total fluorescence)
    P = poly(t,mu,sig)
    Fp = Klcm*P #*c1/(1+P*c1)
    I = np.cumsum(A*Fp)
    return(I)

def I_pair(X, mu, sig, Klcm1, Klcm2):    # Model for dual fluorescence intensity (input=od,time; 
    A,t = X
    XX = (A[0:len(A)/2], t[0:len(t)/2])
                                                    # parameters=mean,std dev,promoter1 affinity,promoter 2 affinity,
    I1 = I(XX,mu,sig,Klcm1)                 # translation rate 1*max transcription rate 1,  
    I2 = I(XX,mu,sig,Klcm2)                 # translation rate 2*max transcription rate 2; 

    return(np.concatenate((I1,I2)))                           # output=thoretical fluorescence intensities 1 & 2)

def dIdt(X,  mu, sig, c1, Klcm):       # Model for fluorescence rate (input=od,time; 
                                       # parameters=mean,std dev,promoter affinity,translation rate*max transcription rate;
    A,t=X                              # output=theoretical fluorescence derivative)
    P = poly(t,mu,sig)
    Fp = Klcm*P*c1/(1+P*c1)
    dIdt = A*Fp
    return(dIdt)

def dIdt_pair(X, mu, sig, c1, c2, Klcm1, Klcm2):    # Model for dual fluorescence rate (input=od,time; 
                                                    # parameters=mean,std dev,promoter1 affinity,promoter 2 affinity,
    dIdt1 = dIdt(X,mu,sig,c1,Klcm1)                 # translation rate 1*max transcription rate 1,  
    dIdt2 = dIdt(X,mu,sig,c2,Klcm2)                 # translation rate 2*max transcription rate 2; 
    return((dIdt1,dIdt2))                           # output=thoretical fluorescence derivatives 1 & 2)
    
def ratio(t,  mu, sig, c1, c2, cm12):

    P = poly(t,mu,sig)
    P = P/np.max(P)
    lr = np.log(cm12*(c1/(1+P*c1))/(c2/(1+P*c2)))
    #dI1dI2 = cm12*(1+P*c2)/(1+P*c1)
    return(lr)

Function that fits the curve to best parameters given initial guesses. curve_fit(function,xdata,ydata, error tolerance, initial guesses)

In [ ]:
datavals = np.concatenate((y-y.min(), c-c.min()))

odin = np.tile(od, 2)
tin = np.tile(t, 2)

zy,czy = curve_fit(I, (od,t), y-y.min(), ftol=1e-6, p0=[20,5,1e3])
zc,czc = curve_fit(I, (od,t), c-c.min(), ftol=1e-6, p0=[20,5,1e3])
zp,czp = curve_fit(I_pair, (odin,tin), datavals, ftol=1e-6, p0=[20,5,1e3,1e3])

In [ ]:
print "Estimated parameters:"
print zp
print "\nCovariance matrix"
print czp
perr = np.sqrt(np.diag(czp))
print "\nStandard error (stdev/mean) for each param:"
print perr/zp

In [ ]:
evalFy=I((od,t),zy[0],zy[1],zy[2])     #evalF=theoretical function with parameters from curve_fit
evalFc=I((od,t),zc[0],zc[1],zc[2])  
evalFp=I_pair((odin,tin),zp[0],zp[1],zp[2],zp[3])
evalFpy = evalFp[0:len(evalFp)/2]
evalFpc = evalFp[len(evalFp)/2:]

plt.figure()
plt.title('fluorescence data vs. model with fitted parameters')
plt.plot(t, evalFy,'.',c='r', alpha=.4,markersize=3,label='Theoretical parameters/function')              
plt.plot(t, y-y.min(),'.',c='m', alpha=.4,markersize=3,label='yfp data')
plt.xlabel('time (hours)')
plt.ylabel('yfp (AU)')
plt.legend()

plt.figure()
plt.title('fluorescence data vs. model with fitted parameters')
plt.plot(t, evalFc,'.',c='b', alpha=.4,markersize=3,label='Theoretical parameters/function')              
plt.plot(t, c-c.min(),'.',c='m', alpha=.4,markersize=3,label='yfp data')
plt.xlabel('time (hours)')
plt.ylabel('yfp (AU)')
plt.legend()

#theoretical polymerase from fitted parameters
plt.figure()
plt.plot(t,poly(t,zy[0],-zy[1]), c='r')
plt.plot(t,poly(t,zc[0],-zc[1]), c='b')
plt.xlabel('time (hours)')
plt.ylabel('[Polymerase] (AU)')

plt.figure()
plt.plot(c-c.min(), y-y.min(), '.', c='k')
plt.plot(evalFc, evalFy, '.', c='m')
plt.plot(evalFpc, evalFpy, '.', c='y')
plt.xlabel('time (hours)')
plt.ylabel('[Polymerase] (AU)')

plt.figure()
plt.title('log(OD)')
plt.plot(t, np.log(od),'g.')